# Predict metrics
This script is responsible create predict metrics

In [1]:
# SET CONSTANTS
INTERVAL_BY_SECONDS_PRED = 60*10
DAYS_TO_PREDICT = 2

In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import explode, from_unixtime, col, to_date, sum, avg, udf, lit, date_trunc, when, max
from pyspark.sql.types import DateType, TimestampType, StructType, StructField, IntegerType, FloatType, StringType
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

import requests
import json
from collections import defaultdict
import locale
import os
import re
from glob import glob
from datetime import datetime, timedelta, date, tzinfo, timezone

DB_URL = "jdbc:postgresql://postgres:5432/themeparkwizard"
PROPERTIES_CUSTOM = {"user": os.environ['POSTGRES_USER'],"password": os.environ['POSTGRES_PASSWORD'], "driver": "org.postgresql.Driver"}

spark = SparkSession.builder \
    .appName("MetricPredict") \
    .config("spark.jars", "jars/postgresql-42.7.7.jar") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .getOrCreate()

Importing plotly failed. Interactive plots will not work.


In [13]:
@udf(returnType=TimestampType())
def min_hour(data):
    if isinstance(data, list):
        for ee in data:
            if ee.type == 'Early Entry':
                return ee.startTime

@udf(returnType=TimestampType())
def max_hour(data):
    if isinstance(data, list):
        for op in data:
            if op.type == 'Operating':
                return op.endTime

def save_prophet_model(model, path):
    with open(path, 'w') as fout:
        fout.write(model_to_json(model))  # Save model
        
def load_prophet_model(path):
    with open(path, 'r') as fin:
        return model_from_json(fin.read())  # Load model

def warm_start_params(m):
    """
    Retrieve parameters from a trained model in the format used to initialize a new Stan model.
    Note that the new Stan model must have these same settings:
        n_changepoints, seasonality features, mcmc sampling
    for the retrieved parameters to be valid for the new model.

    Parameters
    ----------
    m: A trained model of the Prophet class.

    Returns
    -------
    A Dictionary containing retrieved parameters of m.
    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0][0]
        else:
            res[pname] = np.mean(m.params[pname])
    for pname in ['delta', 'beta']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0]
        else:
            res[pname] = np.mean(m.params[pname], axis=0)
    return res

def training_model(path_from_data, attraction, dataframe_att):
    stringify_att = re.sub(r'[ -]*', '', attraction)
    model_filename = path_from_data.rpartition('/')[-1] + '_' + stringify_att
    model_path = 'models/'+model_filename+'.json'
    # newest_model = sorted(glob(model_path+'*'))[-1]
    if os.path.exists(model_path):
        print('Warm start from already trained model!')
        model = load_prophet_model(model_path)
        try:
            model = Prophet().fit(dataframe_att.toPandas(), init=warm_start_params(model))
        except Exception:
            print('Training a complete new model after failling!')
            model = Prophet(growth='flat')#Prophet(holidays=hus_df.toPandas(), growth='flat')
            model.add_country_holidays(country_name='US')
            model.fit(dataframe_att.toPandas())
    else:
        print('Training a complete new model!')
        model = Prophet(growth='flat')#Prophet(holidays=hus_df.toPandas(), growth='flat')
        model.add_country_holidays(country_name='US')
        model.fit(dataframe_att.toPandas())
    save_prophet_model(model, model_path)
    return model

In [14]:
# tz = timezone(timedelta(hours=-3))
# now = datetime.now(tz)
# now = datetime.now()
# start_predict = (now + timedelta(days=1)).date()
start_predict = datetime.now().date()

# limits = df_dl.where("name == 'Guardians of the Galaxy: Cosmic Rewind'")\
#     .select(min_hour(col('operatingHours')).alias('start_time'), max_hour(col('operatingHours')).alias('end_time'))\
#     .limit(1)\
#     .collect()[0]
# limits
# diff_seconds = (limits.end_time - limits.start_time).seconds
# new_day = datetime(start_predict.year, start_predict.month, start_predict.day, limits.start_time.hour, limits.start_time.minute)

new_day = datetime(start_predict.year, start_predict.month, start_predict.day, 0, 0)
# new_day
# data = [(new_day + timedelta(seconds=interval),5,400) for interval in range(0, (limits.end_time - limits.start_time).seconds, INTERVAL_BY_SECONDS_PRED)]
data = [(new_day + timedelta(seconds=interval),5,400) for interval in range(0, 60*60*24*(DAYS_TO_PREDICT+1)+1, INTERVAL_BY_SECONDS_PRED)]
schema = StructType([
    StructField("ds", TimestampType(), False),
    StructField("floor", IntegerType(), False),
    StructField("cap", IntegerType(), False),
])
period_to_predict = spark.createDataFrame(data, schema).cache()

In [15]:
# period_to_predict.show(10)
period_to_predict.select(max('ds').alias('last_date')).collect()[0][0]

datetime.datetime(2025, 8, 20, 0, 0)

In [16]:
schema = StructType([
    StructField("extracted_at_time", TimestampType(), False),
    StructField("wait_time", FloatType(), False),
    StructField("was_predicted", IntegerType(), False),
    StructField("entity_id", StringType(), False)
])
spark.createDataFrame([], schema).write.jdbc(url=DB_URL, table=f"themeparkwizard.predictions_table", mode='overwrite', properties=PROPERTIES_CUSTOM)

for path in glob('datalake_layer/*'):
    path_to_read = path+'/entity_type=attraction/'
    print(path_to_read)
    df = spark.read.orc(path_to_read).cache()

    attractions = [r.name for r in df.select('name').distinct().collect()]
    for att in attractions:
        # df.show(5)
        # att = re.escape(att)
        print(f'Running for {att}')
        df_attraction = df.where(df.name == att)\
                            .where("queue.STANDBY.waitTime is not null")\
                            .withColumn('start_time', min_hour(col('operatingHours')))\
                            .withColumn('end_time', max_hour(col('operatingHours')))\
                            .filter(col('extracted_at_time').between(col('start_time'), col('end_time'))).cache()
        if df_attraction.isEmpty():
            print(f'No data for {att}!')
            continue

        # Get inferior and superir limits for this attraction
        limits = df_attraction.select(max('start_time').alias('start_time'), max('end_time').alias('end_time'))\
                                .limit(1)\
                                .collect()[0]
        # Get entity/attraction id
        entity_id = df_attraction.select('id').limit(1).collect()[0][0]

        print(limits)
        print(entity_id)
        df_attraction = df_attraction.select('extracted_at_time', 'queue.STANDBY.waitTime', 'extracted_at')
        df_attraction.createOrReplaceTempView('current_table')
        df_attraction = spark.sql("""
        WITH enrich_data AS (
            SELECT
                cast(floor(try_divide(extracted_at, 600))*600 as timestamp) as extracted_at_time,
                AVG(waitTime) as waitTime
            FROM current_table
            GROUP BY 1
            UNION ALL
            SELECT
                extracted_at_time,
                waitTime
            FROM current_table
        )
        SELECT 
            ed.*,
            year(extracted_at_time) as year,
            month(extracted_at_time) as month,
            dayofweek(extracted_at_time) as dayofweek,
            hour(extracted_at_time) as hour,
            minute(extracted_at_time) as minute,
            5 as floor,
            400 as cap
        FROM enrich_data ed
        ORDER BY ed.extracted_at_time DESC
        """)

        df_attraction = df_attraction.drop('extracted_at')\
                                    .withColumnRenamed('extracted_at_time', 'ds')\
                                    .withColumnRenamed('waitTime', 'y')
        df_attraction.createOrReplaceTempView('current_table')
        # df_attraction.orderBy('ds', ascending=False).show(25)

        # Build model
        model = training_model(path, att, df_attraction)

        st = datetime(start_predict.year, start_predict.month, start_predict.day, limits.start_time.hour, limits.start_time.minute)
        et = datetime(start_predict.year, start_predict.month, start_predict.day, limits.end_time.hour, limits.end_time.minute)
        if et < st:
            et += timedelta(days=1)
        filter_list_str = []
        for i in range(DAYS_TO_PREDICT):
            print(st, et)
            scol = st.strftime("%Y-%m-%d %H:%M:%S")
            ecol = et.strftime("%Y-%m-%d %H:%M:%S")
            filter_list_str.append(f"ds BETWEEN '{scol}' AND '{ecol}'")
            # period_to_predict = period_to_predict.withColumn(scol, lit(st).cast(TimestampType()))\
            #                                     .withColumn(ecol, lit(et).cast(TimestampType()))\
                                                
            st += timedelta(days=1)
            et += timedelta(days=1)
        period_to_predict_filtered = period_to_predict.where(' OR '.join(filter_list_str))

        predicted_df = spark.createDataFrame(model.predict(period_to_predict_filtered.toPandas()))
        predicted_df.select('ds', 'yhat').show(5)
        predicted_df.createOrReplaceTempView('predicted_table')
        res = spark.sql("""
        SELECT
            ds as extracted_at_time,
            y as wait_time,
            0 as was_predicted
        FROM current_table
        UNION ALL
        SELECT
            ds as extracted_at_time,
            y as wait_time,
            1 as was_predicted
        FROM (SELECT ds, y, ROW_NUMBER() OVER (ORDER BY ds DESC) as rn FROM current_table)
        WHERE rn = 1
        UNION ALL
        SELECT
            ds as extracted_at_time,
            yhat as wait_time,
            1 as was_predicted
        FROM predicted_table
        """).withColumn('entity_id', lit(entity_id))\
            .withColumn('wait_time', when(col('wait_time') <= 5, 5).otherwise(col('wait_time')))
        # res.write.saveAsTable("predictions_table_tmp", partitionBy=['extracted_at_time'], mode='append')
        res.write.option("truncate", True).jdbc(url=DB_URL, table=f"themeparkwizard.predictions_table", mode='append', properties=PROPERTIES_CUSTOM)
        
        # res.select('ds', lit(1).alias('was_predicted'), 'yhat').show(60)
        # break
        
        df_attraction.unpersist()
    # break
    df.unpersist()
# final_df = spark.read.table("predictions_table_tmp").orderBy('extracted_at_time')
# final_df.write.jdbc(url=DB_URL, table=f"themeparkwizard.predictions_table", mode='overwrite', properties=PROPERTIES_CUSTOM)

datalake_layer/animal_kingdom/entity_type=attraction/
Running for Discovery Island Trails
No data for Discovery Island Trails!
Running for Wildlife Express Train
No data for Wildlife Express Train!
Running for Kilimanjaro Safaris
No data for Kilimanjaro Safaris!
Running for Tree of Life
No data for Tree of Life!
Running for Dino-Sue
No data for Dino-Sue!
Running for Wilderness Explorers
No data for Wilderness Explorers!
Running for The Animation Experience at Conservation Station
No data for The Animation Experience at Conservation Station!
Running for Avatar Flight of Passage
Row(start_time=datetime.datetime(2025, 8, 16, 11, 30), end_time=datetime.datetime(2025, 8, 17, 0, 0))
24cf863c-b6ba-4826-a056-0b698989cbf7
Warm start from already trained model!


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/kev1cbgl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/tzyz3eu8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/opt/conda/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92232', 'data', 'file=/tmp/tmpzznmm7qp/kev1cbgl.json', 'init=/tmp/tmpzznmm7qp/tzyz3eu8.json', 'output', 'file=/tmp/tmpzznmm7qp/prophet_model7v_gtrxg/prophet_model-20250817042454.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:24:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:24:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


2025-08-17 11:30:00 2025-08-18 00:00:00
2025-08-18 11:30:00 2025-08-19 00:00:00
+-------------------+-----------------+
|                 ds|             yhat|
+-------------------+-----------------+
|2025-08-17 11:30:00|88.52123899481032|
|2025-08-17 11:40:00| 88.6388811613778|
|2025-08-17 11:50:00|88.72378683882377|
|2025-08-17 12:00:00|88.77980064859197|
|2025-08-17 12:10:00|88.81162920477898|
+-------------------+-----------------+
only showing top 5 rows

Running for Conservation Station
No data for Conservation Station!
Running for Expedition Everest - Legend of the Forbidden Mountain
Row(start_time=datetime.datetime(2025, 8, 16, 11, 30), end_time=datetime.datetime(2025, 8, 17, 0, 0))
64a6915f-a835-4226-ba5c-8389fc4cade3
Warm start from already trained model!


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/vlefqzl8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/87bxa2uo.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/opt/conda/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56360', 'data', 'file=/tmp/tmpzznmm7qp/vlefqzl8.json', 'init=/tmp/tmpzznmm7qp/87bxa2uo.json', 'output', 'file=/tmp/tmpzznmm7qp/prophet_model_9bz0_di/prophet_model-20250817042528.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:25:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:25:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


2025-08-17 11:30:00 2025-08-18 00:00:00
2025-08-18 11:30:00 2025-08-19 00:00:00
+-------------------+-------------------+
|                 ds|               yhat|
+-------------------+-------------------+
|2025-08-17 11:30:00| -6.512341261318468|
|2025-08-17 11:40:00| -5.223776134658024|
|2025-08-17 11:50:00| -3.439328692854133|
|2025-08-17 12:00:00|-1.2673715769473688|
|2025-08-17 12:10:00| 1.1899435330122259|
+-------------------+-------------------+
only showing top 5 rows

Running for DINOSAUR
Row(start_time=datetime.datetime(2025, 8, 16, 11, 30), end_time=datetime.datetime(2025, 8, 17, 0, 0))
55c531b6-3ce2-4c47-a8a1-0dc9107d825b
Warm start from already trained model!


...

2025-08-17 12:30:00 2025-08-18 03:00:00
2025-08-18 12:30:00 2025-08-19 03:00:00
+-------------------+----+
|                 ds|yhat|
+-------------------+----+
|2025-08-17 12:30:00| 5.0|
|2025-08-17 12:40:00| 5.0|
|2025-08-17 12:50:00| 5.0|
|2025-08-17 13:00:00| 5.0|
|2025-08-17 13:10:00| 5.0|
+-------------------+----+
only showing top 5 rows

Running for Enchanted Tales with Belle
No data for Enchanted Tales with Belle!
Running for Haunted Mansion
No data for Haunted Mansion!
Running for Peter Pan's Flight
Row(start_time=datetime.datetime(2025, 8, 16, 12, 30), end_time=datetime.datetime(2025, 8, 17, 3, 0))
86a41273-5f15-4b54-93b6-829f140e5161
Warm start from already trained model!


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/1jz8btcd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/xp6sdyme.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/opt/conda/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=62118', 'data', 'file=/tmp/tmpzznmm7qp/1jz8btcd.json', 'init=/tmp/tmpzznmm7qp/xp6sdyme.json', 'output', 'file=/tmp/tmpzznmm7qp/prophet_model_7371khd/prophet_model-20250817044648.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:46:48 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:46:55 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


2025-08-17 12:30:00 2025-08-18 03:00:00
2025-08-18 12:30:00 2025-08-19 03:00:00
+-------------------+------------------+
|                 ds|              yhat|
+-------------------+------------------+
|2025-08-17 12:30:00| 32.20620902537969|
|2025-08-17 12:40:00|   31.343703953391|
|2025-08-17 12:50:00| 30.76457044971312|
|2025-08-17 13:00:00|30.451080221574834|
|2025-08-17 13:10:00| 30.38166095853152|
+-------------------+------------------+
only showing top 5 rows

Running for Prince Charming Regal Carrousel
Row(start_time=datetime.datetime(2025, 8, 16, 12, 30), end_time=datetime.datetime(2025, 8, 17, 3, 0))
273ddb8d-e7b5-4e34-8657-1113f49262a5
Warm start from already trained model!


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/7d_79uix.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzznmm7qp/tgruykhd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/opt/conda/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=53929', 'data', 'file=/tmp/tmpzznmm7qp/7d_79uix.json', 'init=/tmp/tmpzznmm7qp/tgruykhd.json', 'output', 'file=/tmp/tmpzznmm7qp/prophet_modelyl8bzlfm/prophet_model-20250817044721.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:47:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:47:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


2025-08-17 12:30:00 2025-08-18 03:00:00
2025-08-18 12:30:00 2025-08-19 03:00:00
+-------------------+------------------+
|                 ds|              yhat|
+-------------------+------------------+
|2025-08-17 12:30:00| 5.715218851632872|
|2025-08-17 12:40:00| 5.893784217355363|
|2025-08-17 12:50:00|6.0089678669448805|
|2025-08-17 13:00:00| 6.075650536231043|
|2025-08-17 13:10:00| 6.107989133946711|
+-------------------+------------------+
only showing top 5 rows

Running for Trick-or-Treat Locations at Disney After Hours Boo Bash
No data for Trick-or-Treat Locations at Disney After Hours Boo Bash!
datalake_layer/universal_epic_universe/entity_type=attraction/
Running for Fyre Drill
No data for Fyre Drill!
Running for Yoshi's Adventure™
No data for Yoshi's Adventure™!
Running for Mine-Cart Madness™
No data for Mine-Cart Madness™!
Running for Harry Potter and the Battle at the Ministry™
No data for Harry Potter and the Battle at the Ministry™!
Running for Dragon Racer's Rally
No da

In [14]:
# Finish session
spark.stop()

In [12]:
%pip list

Package                       Version
----------------------------- ------------
alembic                       1.12.0
altair                        5.1.2
anyio                         4.0.0
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.3.0
asttokens                     2.4.0
async-generator               1.10
async-lru                     2.0.4
attrs                         23.1.0
Babel                         2.13.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
beautifulsoup4                4.12.2
bleach                        6.1.0
blinker                       1.6.3
bokeh                         3.3.0
boltons                       23.0.0
Bottleneck                    1.3.7
Brotli                        1.1.0
cached-property               1.5.2
certifi                       2023.7.22
certipy                       0.1.3
cffi                          1.16.0
charset-normalizer            3.3.0
click   

In [12]:
# df_dl = spark.read.orc('datalake_layer/epcot')

# # df_dl.printSchema()

In [13]:
# df_dl.createOrReplaceTempView('datalake_table')
# test_df = spark.sql("""
# SELECT 
#     extracted_date,
#     name AS attraction_name, 
#     AVG(queue.STANDBY.waitTime) AS avg_standby_waittime
# FROM datalake_table
# WHERE entity_type = 'ATTRACTION' AND queue.STANDBY.waitTime is not null
# GROUP BY 1, 2
# ORDER BY 1
# """)
# test_df.printSchema()
# test_df.write.jdbc(url=DB_URL, table="themeparkwizard.agg_avg_time_epcot", mode='overwrite', properties=PROPERTIES_CUSTOM)

In [14]:
# INTERVAL_OF_MINUTES = 60 * 15

In [15]:
# rest_df = spark.sql(f"""
# WITH wait_by_party AS (
#     SELECT
#         extracted_date,
#         date_format(cast(floor(try_divide(extracted_at, {INTERVAL_OF_MINUTES}))*{INTERVAL_OF_MINUTES} as timestamp), 'HH:mm') as time_of_the_day, --extracted_date,
#         name AS attraction_name,
#         CASE 
#         WHEN da_exp.partySize <= 2 THEN
#             'Small group (<= 2)'
#         WHEN da_exp.partySize > 2 AND da_exp.partySize <= 4 THEN
#             'Medium group (3 and 4)'
#         WHEN da_exp.partySize > 4 AND da_exp.partySize <= 6 THEN
#             'Medium group (5 ant 6)'
#         WHEN da_exp.partySize > 6 THEN
#             'Big group (> 6)'
#         END as party_size,
#         COALESCE(AVG(da_exp.waitTime), 0) as avg_wait_time,
#         STDDEV(da_exp.waitTime) AS stddev_wait_time
#     FROM datalake_table
#     LATERAL VIEW EXPLODE(diningAvailability) as da_exp
#     WHERE entity_type = 'SHOW' --AND name = 'Garden Grill Restaurant'
#     GROUP BY 1,2,3,4
# )
# SELECT 
#     *
# FROM wait_by_party
# ORDER BY 1, 2, 3, 4
# """).show(50, truncate=False)

In [16]:
# rest_df.printSchema()
# rest_df.write.jdbc(url=DB_URL, table="themeparkwizard.restaurant_wait_time_epcot", mode='overwrite', properties=PROPERTIES_CUSTOM)